<a href="https://colab.research.google.com/github/cgrundman/ZTM-TensorFlow-Developer/blob/main/08_nlp_fundamentals_in_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP Fundamentals in TensorFlow

NLP has the goal of deriving information out of natural language (could be sequences text or speech).

Another common term for NLP problems is sequence to sequence (seq2seq) problems.

## Check for GPU

In [1]:
!nvidia-smi -L

/bin/bash: line 1: nvidia-smi: command not found


## Get helper functions

In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

# Import series of helper functions for the notebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2024-07-19 15:42:34--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2024-07-19 15:42:34 (55.2 MB/s) - ‘helper_functions.py’ saved [10246/10246]



## Get a text dataset

The dataset we're going to be using is Kaggle's introduction to NLP dataset (text samples of Tweets labelled as disater or not disaster)

See the original here: https://www.kaggle.com/competitions/nlp-getting-started

In [3]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

# Unzip data
unzip_data("nlp_getting_started.zip")

--2024-07-19 15:42:48--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.196.207, 173.194.216.207, 108.177.11.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.196.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.01s   

2024-07-19 15:42:48 (45.3 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## Visualizing a text dataset

To visualize our text samples, we first have to read them in. One way to do so would be to read them in: https://realpython.com/read-write-files-python/

Another way to do this is to use Pandas...

In [4]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [6]:
# What does the test dataframe look like?
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [7]:
# How many examples of each class?
train_df.target.value_counts()

target
0    4342
1    3271
Name: count, dtype: int64

In [8]:
# How many total samples?
len(train_df), len(test_df)

(7613, 3263)

In [9]:
# Lets visualize some random training examples
import random
random_index = random.randint(1, len(train_df)-5) # create random indexes not higher than the total number of samples
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real disastor)" if target > 0 else "(not real disastor)")
  print(f"Text:\n{text}\n")
  print("---\n")

Target: 0 (not real disastor)
Text:
@Chrisman528 fatality ...

---

Target: 0 (not real disastor)
Text:
New on Ebay UK Star Wars Storm Trooper Pop! Vinyl Bobble Head Figure POP Funko http://t.co/KJbXIeypma http://t.co/ENPjCfMa8L

---

Target: 0 (not real disastor)
Text:
http://t.co/MoA0q0AuFa Jacksonville family bands together as memorial is planned for toddler who ... - FloridaÛ_ http://t.co/NKOu7zWwRT

---

Target: 0 (not real disastor)
Text:
Hibernating pbx irrespective of pitch fatality careerism pan: crbZFZ

---

Target: 1 (real disastor)
Text:
Who is bringing the tornadoes and floods. Who is bringing the climate change. God is after America He is plaguing her
 
#FARRAKHAN #QUOTE

---



### Split data into training and validation

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
# Use train_test_split to split training data into training and validation set
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1, # use 10% of training data for validation data
                                                                            random_state=42)

In [12]:
# Check the lengths
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [13]:
# Check the first 10 samples
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

## Converting text into numbers

When dealing with a text problem, one of the first things you'll have to do before you can build a model is to convert your text to numbers.

There are a few ways to do this, namely:
* Tokenization - direct mapping of token (a token could be a word or a character) to number
* Embedding - create a matrix of feature vector for eah token (the size of the feature vector can be defined and this embedding can be learned)

### Text Vectorization (tokenization)

In [14]:
train_sentences[:5]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
      dtype=object)

In [15]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Use the default TextVectorization parameters
text_vectorizer = TextVectorization(max_tokens=None, # how many word in our tokenization (automatically add <OOV>)
                                    standardize='lower_and_strip_punctuation',
                                    split="whitespace",
                                    ngrams=None, # create grouos of n-words?
                                    output_mode='int', # how to map tokens to number
                                    output_sequence_length=None, # How long do you want your sequences to be
                                    pad_to_max_tokens=False)

In [16]:
# Find the average number of tokens (words) in the training tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [17]:
# Setup text vectorization variables
max_vocab_length = 10000 # max number of words in ourvocabulary
max_length = 15 # max length of our sequences

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [18]:
# Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)

In [19]:
# Create a sample sentensce and tokenize it
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [20]:
# Choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\
        \n\nVectorized version:")
text_vectorizer([random_sentence])

Original text:
 If you're reading this go accidentally fall off a cliff mate        

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[  47,  172, 1262,   19,  112, 2203,  278,  102,    3,  403, 3650,
           0,    0,    0,    0]])>

In [21]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary() # get all of the unique words in our vocabulary
top_5_words = words_in_vocab[:5] # get the most common words
bottom_5_words = words_in_vocab[-5:] # get the least 5 common words
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"5 most common words: {top_5_words}")
print(f"5 least common words: {bottom_5_words}")

Number of words in vocab: 10000
5 most common words: ['', '[UNK]', 'the', 'a', 'in']
5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


### Creating an Embedding using an Embedding Layer

TO make our embedding, we're going to use TensorFlow's embedding layer: https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding

The parameters we care most about for our embedding layer:
* `input_dim` - the size of our vocabulary
* `output_dim` - the size of the ouput embedding vector, for example, a value of 100 would mean each token gets represented by a vector 100 long
* `input_length` - length of the sequences being passed to the embedding layer

In [22]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length, # set input shape
                             output_dim=128,
                             input_length=max_length # how long is each input
                             )

embedding

In [23]:
# Get a random sentence from the training set
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\
        \n\nEmbedded version:")

# Embed the random sentence (turn it into dense vectors of fixed size)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
 '[+54 -9] How do people not know who Kendall Jenner is? She has 6 times the Instagram followers of GD' SCREAMING LMAO        

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.02868735,  0.04411114, -0.0479099 , ..., -0.03628539,
          0.03411878,  0.02910629],
        [-0.03661414,  0.03416497, -0.03952327, ...,  0.02818275,
         -0.01221027, -0.02781491],
        [-0.01445321, -0.00476816,  0.0432543 , ..., -0.04648093,
          0.04817145, -0.00669616],
        ...,
        [ 0.02958668, -0.03412383, -0.04941463, ...,  0.0229677 ,
         -0.04044069,  0.04655478],
        [-0.00304214, -0.00792438, -0.01124758, ...,  0.04481543,
          0.02304326,  0.00806961],
        [-0.03331125, -0.03598884,  0.01034145, ...,  0.00175822,
         -0.01373843,  0.01869341]]], dtype=float32)>

In [24]:
# Check out a single token's embedding
sample_embed[0][0], sample_embed[0][0].shape, random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-2.86873467e-02,  4.41111363e-02, -4.79099043e-02,  7.08412006e-03,
        -3.52866650e-02,  3.91519777e-02,  4.71291654e-02, -2.39123590e-02,
         4.32246663e-02,  1.68117397e-02, -2.16296911e-02, -2.52826698e-02,
        -4.93841432e-02,  3.48609798e-02,  2.01799758e-02, -2.66513359e-02,
         3.60128172e-02, -4.46850918e-02,  6.76155090e-03,  4.15362455e-02,
        -3.09864637e-02,  1.14589222e-02, -2.50590201e-02, -2.74392851e-02,
         4.67964448e-02, -4.46409471e-02, -4.69425917e-02, -4.64378707e-02,
         1.36772506e-02,  2.02363990e-02, -3.40736285e-02, -1.05514303e-02,
         1.09307058e-02, -9.53353569e-03,  1.70258023e-02, -2.40240581e-02,
        -1.47629976e-02, -4.30487990e-02,  4.40197103e-02, -1.45411268e-02,
        -2.14324128e-02, -2.64818799e-02,  1.61172189e-02, -6.28687069e-03,
         3.13513987e-02, -3.78590710e-02, -2.72608921e-03,  3.82431187e-02,
        -2.83270832e-02, -3.82836461e-0

## Modelling a text dataset (running a series of experiments)

Now we've got a way to turn out text sequences into numbers, it's time to start builging a series of modelling experiments.

We'll start with a baseline and move on from there.

* Model 0: Naive Bayes (baseline), this is from SKLearn ML map: https://scikit-learn.org/1.3/tutorial/machine_learning_map/index.html
* Model 1: Feed-Forward neural network (dense model)
* Model 2: LSTM model (RNN)
* Model 3: GRU model (RNN)
* Model 4: Bidirectionsl-LSTM model (RNN)
* Model 5: 1D Convolutional Neural Network (CNN)
* Model 6: TensorFlow Hub Pretrained Feature Extractor (using transfer learning for NLP)
* Model 7: Same as model 6 with 10% of training data

How are we going to approach all of these?

Use the standard steps in modelling with TensorFlow:

* Create a model
* Build a model
* Fit a model
* Evaluate our model

### Model 0: Getting a baseline

As with all machine learning experiment, it's important to create a baseline model so your've got a benchmark for future experiments to build upon.

To create our baseline, we'll use Sklearn's Multinomial Naive Bayes using the TF-IFDF formula to convert our words to numbers.

**Note:** It's common practice to use non-DL algorithms as a baseline because of their speed and then later using DL to see if you can improve upon them.

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline
model_0 = Pipeline([
    ("tfidf", TfidfVectorizer()), # convert words to numbers
    ("clf", MultinomialNB()) # model the text
])

# Fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [26]:
# Evaluate our baseline model
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Out baseline model achieves ana accuracy of: {baseline_score*100:.2f}%")

Out baseline model achieves ana accuracy of: 79.27%


In [27]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

### Creating an evaluation function for our model experimnets

We could evaluate all of our model's predictions with different metrics every time, however this will be cumbersome and will easily be fixed with a function.

Let's create one to compare our model's predictions with the truth labels using the following metrics:
* Accuracy
* Precision
* Recall
* F1 score

For a deep overview of many different evaluation methods, see the Sklearn documnetation: https://scikit-learn.org/stable/modules/model_evaluation.html

In [28]:
# Function to evaluate: accuracy, precision, recall, f1_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def calculate_results(y_true, y_pred):
  """
  Calculate model accuracy, precision, recall, and f1 score of a binary classification model_0
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision
  model_precision = precision_score(y_true, y_pred) * 100
  # Calculate recall
  model_recall = recall_score(y_true, y_pred) * 100
  # Calculate f1_score
  model_f1 = f1_score(y_true, y_pred) * 100

  model_results = {
      "accuracy": model_accuracy,
      "precision": model_precision,
      "recall": model_recall,
      "f1": model_f1
  }

  return model_results

In [29]:
# Get Baseline Results
baseline_results = calculate_results(y_true=val_labels, y_pred=baseline_preds)

baseline_results

{'accuracy': 79.26509186351706,
 'precision': 88.6178861788618,
 'recall': 62.643678160919535,
 'f1': 73.4006734006734}

### Model 1: A simple dense model

In [30]:
# Create a tensorboard callback (need to create a new one for each model)
from helper_functions import create_tensorboard_callback

# Create a directory to save TensorBoard logs
SAVE_DIR = "model_logs"

In [31]:
# Build model with the Functional API
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype=tf.string) # inputs are 1-dimensional strings
x = text_vectorizer(inputs) # turn the input text into numbers
x = embedding(x) # create an embedding of the numberized inputs
x = layers.GlobalAveragePooling1D()(x) # condense the feature vector for each token to one vector
outputs = layers.Dense(1, activation="sigmoid")(x)
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense")

In [32]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (  (None, 128)               0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1280129 (4.88 MB)
Trainable params: 128

In [33]:
# Compile model
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [34]:
# FIt the model
model_1_history = model_1.fit(x=train_sentences,
                              y=train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                     experiment_name="model_1_dense")])

Saving TensorBoard log files to: model_logs/model_1_dense/20240719-154252
Epoch 1/5
215/215 [==============================] - 15s 50ms/step - loss: 0.6123 - accuracy: 0.6930 - val_loss: 0.5370 - val_accuracy: 0.7546
Epoch 2/5
215/215 [==============================] - 8s 36ms/step - loss: 0.4419 - accuracy: 0.8148 - val_loss: 0.4689 - val_accuracy: 0.7861
Epoch 3/5
215/215 [==============================] - 9s 43ms/step - loss: 0.3477 - accuracy: 0.8602 - val_loss: 0.4583 - val_accuracy: 0.7953
Epoch 4/5
215/215 [==============================] - 10s 47ms/step - loss: 0.2847 - accuracy: 0.8923 - val_loss: 0.4728 - val_accuracy: 0.7913
Epoch 5/5
215/215 [==============================] - 8s 36ms/step - loss: 0.2381 - accuracy: 0.9108 - val_loss: 0.4812 - val_accuracy: 0.7822


In [35]:
# Check the results
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 2ms/step - loss: 0.4812 - accuracy: 0.7822


[0.48123592138290405, 0.7821522355079651]

In [36]:
# Make some predictions and evaluate those
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs.shape

24/24 [==============================] - 0s 3ms/step


(762, 1)

In [37]:
# Convert model prediction probabilities to label format
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [38]:
# Calcualate out model_1 results
model_1_results = calculate_results(y_true=val_labels,
                                    y_pred=model_1_preds)
model_1_results

{'accuracy': 78.21522309711287,
 'precision': 81.59722222222221,
 'recall': 67.52873563218391,
 'f1': 73.89937106918238}

In [39]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 88.6178861788618,
 'recall': 62.643678160919535,
 'f1': 73.4006734006734}

In [40]:
import numpy as np
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([False, False,  True,  True])

## Visualizing learned embeddings

In [41]:
# Get the vocabulary from the text vectorization layer
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [42]:
# Model 1 summary
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (  (None, 128)               0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1280129 (4.88 MB)
Trainable params: 128

In [43]:
# Get the weight matrix of embedding layer
# (thesea re the numerical reresentations of each token in our training data which have been trained for 5 epochs)
embed_weights = model_1.get_layer("embedding").get_weights()[0]
print(embed_weights.shape)

(10000, 128)


Now we've got the embedding matrix our model has learned to represent our tokens, let's see how we can visualize it.

To do so, TensorFlow has a handy tool called projector: https://projector.tensorflow.org/

And TensorFLow also has an incredible guide on word embeddings themselves: https://www.tensorflow.org/text/guide/word_embeddings

In [44]:
# # Create embedding files (we got thisfrom TensorFlow's word embeddings documentation)
# import io
# out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
# out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

# for index, word in enumerate(words_in_vocab):
#   if index == 0:
#     continue  # skip 0, it's padding.
#   vec = embed_weights[index]
#   out_v.write('\t'.join([str(x) for x in vec]) + "\n")
#   out_m.write(word + "\n")
# out_v.close()
# out_m.close()

In [45]:
# # Download files from Colab to upload to projector
# try:
#   from google.colab import files
#   files.download('vectors.tsv')
#   files.download('metadata.tsv')
# except Exception:
#   pass

## Recurrent Neural Networks (RNN's)

RNN's are useful for sequence data.

The premise of a recurrent neural network is to use the representation of a previous input to aid the representation of a later input.

**Resources:** If you want an overview of the internals of a recurrent neural network, see the following:
* MIT's sequence modeling lecture: https://youtu.be/CznICCPa63Q?si=4oY5gEz3NRP7ftK5
* Chris Olah's intro to LSTM's: https://colah.github.io/posts/2015-08-Understanding-LSTMs/
* Andrei Karpathy's the unreasonable effectiveness of recurrent neural networks: https://karpathy.github.io/2015/05/21/rnn-effectiveness/

### Model 2: LSTM

LSTM = long short term memory (one of the most popular LSTM cells)

Our structure of an RNN typically looks like this:

```
Input (text) -> Tokenize -> Embedding -> Layres (RNNs/dense) -> Output (label probability)
```

In [46]:
# Create an LSTM model
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
# print(x.shape)
# x = layers.LSTM(64, return_sequences=True)(x) # when you're stacking RNN cells together, you need to set return_sequences=True
# print(x.shape)
x = layers.LSTM(64)(x)
# print(x.shape)
# x = layers.Dense(64, activation="relu")(x)
# print(x.shape)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_2 = tf.keras.Model(inputs, outputs, name="model_2_LSTM")

In [47]:
# Get a summary
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1329473 (5.07 MB)
Trainable params: 1329473 (5.07 MB)
Non-trainable params: 0 (0.00 Byte)
________________

In [48]:
# Compile the model
model_2.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [49]:
# Fit the model
model_2_history = model_2.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                    "model_2_LSTM")])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20240719-154420
Epoch 1/5
215/215 [==============================] - 17s 56ms/step - loss: 0.2177 - accuracy: 0.9193 - val_loss: 0.5312 - val_accuracy: 0.7808
Epoch 2/5
215/215 [==============================] - 14s 66ms/step - loss: 0.1587 - accuracy: 0.9412 - val_loss: 0.6081 - val_accuracy: 0.7887
Epoch 3/5
215/215 [==============================] - 12s 55ms/step - loss: 0.1263 - accuracy: 0.9526 - val_loss: 0.7501 - val_accuracy: 0.7835
Epoch 4/5
215/215 [==============================] - 13s 59ms/step - loss: 0.1040 - accuracy: 0.9585 - val_loss: 0.8512 - val_accuracy: 0.7743
Epoch 5/5
215/215 [==============================] - 10s 45ms/step - loss: 0.0819 - accuracy: 0.9653 - val_loss: 0.9466 - val_accuracy: 0.7756


In [50]:
# Make predictions with LSTM model
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

24/24 [==============================] - 1s 9ms/step


array([[5.7600457e-02],
       [7.1879941e-01],
       [9.9974298e-01],
       [1.6367665e-02],
       [2.8007638e-04],
       [9.9850059e-01],
       [9.0103149e-01],
       [9.9984384e-01],
       [9.9972576e-01],
       [4.1259837e-01]], dtype=float32)

In [51]:
# Convert model 2 pred probs to lables
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [52]:
# Calculate model 2 results
model_2_results = calculate_results(y_true=val_labels,
                                    y_pred=model_2_preds)
model_2_results

{'accuracy': 77.55905511811024,
 'precision': 79.59866220735786,
 'recall': 68.39080459770115,
 'f1': 73.57032457496135}

In [53]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 88.6178861788618,
 'recall': 62.643678160919535,
 'f1': 73.4006734006734}

### Model 3: GRU

Another popular and effective RNN component is the GRU or gated recurrent unit.

The GRU cell has similar features to an LSTM cell but has less parameters.

In [54]:
# Create an GRU model
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GRU(64)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_3 = tf.keras.Model(inputs, outputs, name="model_3_GRU")

In [55]:
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 gru (GRU)                   (None, 64)                37248     
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1317313 (5.03 MB)
Trainable params: 1317313 (5.03 MB)
Non-trainable params: 0 (0.00 Byte)
_________________

In [56]:
# Compile the model
model_3.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [57]:
# Fit the model
model_3_history = model_3.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                    "model_3_GRU")])

Saving TensorBoard log files to: model_logs/model_3_GRU/20240719-154548
Epoch 1/5
215/215 [==============================] - 19s 52ms/step - loss: 0.1581 - accuracy: 0.9361 - val_loss: 0.7158 - val_accuracy: 0.7848
Epoch 2/5
215/215 [==============================] - 14s 64ms/step - loss: 0.0864 - accuracy: 0.9695 - val_loss: 0.8208 - val_accuracy: 0.7677
Epoch 3/5
215/215 [==============================] - 12s 55ms/step - loss: 0.0721 - accuracy: 0.9702 - val_loss: 0.8806 - val_accuracy: 0.7756
Epoch 4/5
215/215 [==============================] - 11s 50ms/step - loss: 0.0566 - accuracy: 0.9761 - val_loss: 1.0302 - val_accuracy: 0.7782
Epoch 5/5
215/215 [==============================] - 9s 40ms/step - loss: 0.0556 - accuracy: 0.9766 - val_loss: 1.0884 - val_accuracy: 0.7664


In [58]:
# Make predictions with GRU model
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs[:10]

24/24 [==============================] - 3s 7ms/step


array([[1.4495087e-01],
       [7.1993345e-01],
       [9.9984211e-01],
       [1.7015763e-01],
       [1.5570181e-04],
       [9.9937862e-01],
       [8.4033889e-01],
       [9.9993277e-01],
       [9.9988151e-01],
       [8.3823091e-01]], dtype=float32)

In [59]:
# Convert model 3 pred probs to labels
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [60]:
# Calculate model 2 results
model_3_results = calculate_results(y_true=val_labels,
                                    y_pred=model_3_preds)
model_3_results

{'accuracy': 76.64041994750657,
 'precision': 76.3975155279503,
 'recall': 70.6896551724138,
 'f1': 73.43283582089552}

In [61]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 88.6178861788618,
 'recall': 62.643678160919535,
 'f1': 73.4006734006734}

### Model 4: Bidirectional RNN

Normal RNN's go from lft ot right (just like you'd read an English sentence) however, a biderectional RNN goes from right left as well as left to right.

In [62]:
# Create a Biderectional model
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
# x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_4 = tf.keras.Model(inputs, outputs, name="model_4_Birectional")

In [63]:
# Get a summary
model_4.summary()

Model: "model_4_Birectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 bidirectional (Bidirection  (None, 128)               98816     
 al)                                                             
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1378945 (5.26 MB)
Trainable param

In [64]:
# Complie the model
model_4.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [65]:
# Fit the model
model_4_history = model_4.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     "model_4_bidirectional")])

Saving TensorBoard log files to: model_logs/model_4_bidirectional/20240719-154656
Epoch 1/5
215/215 [==============================] - 26s 82ms/step - loss: 0.1023 - accuracy: 0.9702 - val_loss: 1.0905 - val_accuracy: 0.7690
Epoch 2/5
215/215 [==============================] - 9s 41ms/step - loss: 0.0506 - accuracy: 0.9764 - val_loss: 1.4649 - val_accuracy: 0.7769
Epoch 3/5
215/215 [==============================] - 10s 47ms/step - loss: 0.0470 - accuracy: 0.9780 - val_loss: 1.4035 - val_accuracy: 0.7717
Epoch 4/5
215/215 [==============================] - 10s 47ms/step - loss: 0.0437 - accuracy: 0.9806 - val_loss: 1.4759 - val_accuracy: 0.7690
Epoch 5/5
215/215 [==============================] - 8s 38ms/step - loss: 0.0409 - accuracy: 0.9796 - val_loss: 1.3719 - val_accuracy: 0.7743


In [66]:
# Make predictions with our bidirectional model
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:10]

24/24 [==============================] - 1s 8ms/step


array([[7.4746826e-04],
       [6.4895529e-01],
       [9.9998623e-01],
       [3.0449834e-01],
       [2.2824059e-05],
       [9.9984539e-01],
       [5.7420301e-01],
       [9.9999177e-01],
       [9.9998778e-01],
       [9.8624283e-01]], dtype=float32)

In [67]:
# Convert pred propbs to pred labels
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [68]:
# Calculat ethe resuts of out bidirectional model
model_4_results = calculate_results(y_true=val_labels,
                                    y_pred=model_4_preds)
model_4_results

{'accuracy': 77.42782152230971,
 'precision': 78.94736842105263,
 'recall': 68.96551724137932,
 'f1': 73.61963190184049}

In [69]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 88.6178861788618,
 'recall': 62.643678160919535,
 'f1': 73.4006734006734}

## Convolutional Neural Networks for Text (and other types of sequences)

We'veused CNNs for images but images are typically 2d (height x width)... however, our text data id 1D.

Previously we've used Conv2D for our image data but now we're going to use Conv1D.

THe tyical structure of a Conv1D model for sequences (in our case text):

```
Inputs (text) -> Tokenization -> Embedding -> Layer(s) (typically Conv1D + pooling) -> Outputs (class predictions)
```

### Model 5: Conv1D

For different explaations of parameters see:
* https://poloclub.github.io/cnn-explainer/ (this is for 2D but can relate to 1D data)
* Difference between "same" and "valid" padding: https://stackoverflow.com/questions/37674306/what-is-the-difference-between-same-and-valid-padding-in-tf-nn-max-pool-of-t

In [70]:
# Test out our embedding layer, Conv1D layer and max pooling
from tensorflow.keras import layers
embedding_test = embedding(text_vectorizer(["thisis a test sentence"]))
conv_1d = layers.Conv1D(filters=32,
                        kernel_size=5,
                        activation="relu",
                        padding="same")
conv_1d_output = conv_1d(embedding_test)
max_pool = layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d_output)

embedding_test.shape, conv_1d_output.shape, max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 15, 32]), TensorShape([1, 32]))

In [71]:
# embedding_test

In [72]:
# conv_1d_output

In [73]:
# max_pool_output

In [78]:
# Create 1-dimensional convolutional layer to model sequences
from tensorflow.keras import layers
imputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Conv1D(filters=64, kernel_size=5, strides=1, activation="relu", padding="valid")(x)
x = layers.GlobalMaxPool1D()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_5 = tf.keras.Model(inputs, outputs, name="model_5_Conv1D")

# Compile Conv1D
model_5.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

# Get a summary of our Conv1D model
model_5.summary()

Model: "model_5_Conv1D"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 conv1d_2 (Conv1D)           (None, 11, 64)            41024     
                                                                 
 global_max_pooling1d_2 (Gl  (None, 64)                0         
 obalMaxPooling1D)                                               
                                                                 
 dense_4 (Dense)             (None, 1)              

In [79]:
# Fit the model
model_5_history = model_5.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     "Conv1D")])

Saving TensorBoard log files to: model_logs/Conv1D/20240719-155140
Epoch 1/5
215/215 [==============================] - 10s 33ms/step - loss: 0.1198 - accuracy: 0.9634 - val_loss: 0.9203 - val_accuracy: 0.7651
Epoch 2/5
215/215 [==============================] - 8s 36ms/step - loss: 0.0737 - accuracy: 0.9734 - val_loss: 1.1054 - val_accuracy: 0.7703
Epoch 3/5
215/215 [==============================] - 5s 24ms/step - loss: 0.0619 - accuracy: 0.9746 - val_loss: 1.1179 - val_accuracy: 0.7520
Epoch 4/5
215/215 [==============================] - 5s 23ms/step - loss: 0.0549 - accuracy: 0.9778 - val_loss: 1.1684 - val_accuracy: 0.7441
Epoch 5/5
215/215 [==============================] - 7s 35ms/step - loss: 0.0506 - accuracy: 0.9788 - val_loss: 1.2052 - val_accuracy: 0.7507


In [80]:
# Make some predictions with our Conv1D model
model_5_pred_probs = model_5.predict(val_sentences)
model_5_pred_probs[:10]

24/24 [==============================] - 0s 5ms/step


array([[2.7225530e-01],
       [9.7871828e-01],
       [9.9995852e-01],
       [8.1250466e-02],
       [8.3764627e-07],
       [9.9902999e-01],
       [9.6267146e-01],
       [9.9998116e-01],
       [9.9999976e-01],
       [9.1777658e-01]], dtype=float32)

In [81]:
# Convert model 5 pred probs to labels
model_5_preds = tf.squeeze(tf.round(model_5_pred_probs))
model_5_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [82]:
# Evaluate model 5 predictions
model_5_results = calculate_results(y_true=val_labels,
                                    y_pred=model_5_preds)
model_5_results

{'accuracy': 75.06561679790026,
 'precision': 73.93939393939394,
 'recall': 70.11494252873564,
 'f1': 71.97640117994099}

### Model 6: TensorFlow Hub Pretrained Sentence Encoder

Now we've built a few of our own models, let's try and use transfer learning for NLP, specifically using TensorFlow Hub's Universal Sentence Encoder: https://www.kaggle.com/models/google/universal-sentence-encoder/TensorFlow2/universal-sentence-encoder/2

In [86]:
sample_sentence

"There's a flood in my street!"

In [85]:
import tensorflow_hub as hub
import tensorflow as tf
embed = hub.load("https://www.kaggle.com/models/google/universal-sentence-encoder/TensorFlow2/universal-sentence-encoder/2")
embed_samples = embed([
    sample_sentence,
    "When you call the univeral sentence encoder on a sentence, it turns it into  numbers."])
print(embed_samples[0][:50])

tf.Tensor(
[-0.01157025  0.02485911  0.02878051 -0.012715    0.03971541  0.08827761
  0.02680988  0.05589838 -0.01068731 -0.00597293  0.00639321 -0.01819516
  0.00030816  0.09105889  0.05874645 -0.03180629  0.01512474 -0.05162925
  0.00991366 -0.06865345 -0.04209306  0.0267898   0.03011009  0.00321065
 -0.00337968 -0.04787356  0.0226672  -0.00985927 -0.04063615 -0.01292093
 -0.04666382  0.05630299 -0.03949255  0.00517682  0.02495827 -0.07014439
  0.0287151   0.0494768  -0.00633978 -0.08960193  0.02807119 -0.00808364
 -0.01360601  0.05998649 -0.10361788 -0.05195372  0.00232958 -0.02332531
 -0.03758106  0.03327729], shape=(50,), dtype=float32)


In [87]:
embed_samples

<tf.Tensor: shape=(2, 512), dtype=float32, numpy=
array([[-0.01157025,  0.02485911,  0.02878051, ..., -0.00186124,
         0.02315822, -0.01485021],
       [ 0.03680136, -0.09122784, -0.00470381, ..., -0.03107636,
        -0.00511169, -0.00524487]], dtype=float32)>

In [88]:
# Create a Keras Layer using the USE pretrained layer from tensorflow hub
sentence_encoder_layer = hub.KerasLayer("https://www.kaggle.com/models/google/universal-sentence-encoder/TensorFlow2/universal-sentence-encoder/2",
                                        input_shape=[],
                                        dtype=tf.string,
                                        trainable=False,
                                        name="USE")

In [97]:
# Create a model using the Sequential API
model_6 = tf.keras.Sequential([
    sentence_encoder_layer,
    layers.Dense(64, activation="relu"),
    layers.Dense(1, activation="sigmoid", name="output_layer")
], name="model_6_USE")

# Compile
model_6.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

# Get a summary
model_6.summary()

Model: "model_6_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_8 (Dense)             (None, 64)                32832     
                                                                 
 output_layer (Dense)        (None, 1)                 65        
                                                                 
Total params: 256830721 (979.73 MB)
Trainable params: 32897 (128.50 KB)
Non-trainable params: 256797824 (979.61 MB)
_________________________________________________________________


In [98]:
# Train a classifier on top of USE pretrained embeddings
model_6_history = model_6.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     "tf_hub_sentence_encoder")])

Saving TensorBoard log files to: model_logs/tf_hub_sentence_encoder/20240719-164211
Epoch 1/5
215/215 [==============================] - 5s 15ms/step - loss: 0.5074 - accuracy: 0.7854 - val_loss: 0.4533 - val_accuracy: 0.7927
Epoch 2/5
215/215 [==============================] - 3s 13ms/step - loss: 0.4158 - accuracy: 0.8152 - val_loss: 0.4403 - val_accuracy: 0.8058
Epoch 3/5
215/215 [==============================] - 3s 15ms/step - loss: 0.4014 - accuracy: 0.8221 - val_loss: 0.4340 - val_accuracy: 0.8110
Epoch 4/5
215/215 [==============================] - 4s 18ms/step - loss: 0.3932 - accuracy: 0.8254 - val_loss: 0.4296 - val_accuracy: 0.8189
Epoch 5/5
215/215 [==============================] - 3s 13ms/step - loss: 0.3863 - accuracy: 0.8282 - val_loss: 0.4293 - val_accuracy: 0.8136


In [99]:
# Make predictions with USE TF Hub Model
model_6_pred_probs = model_6.predict(val_sentences)
model_6_pred_probs[:10]

24/24 [==============================] - 1s 10ms/step


array([[0.14286274],
       [0.78954756],
       [0.98854893],
       [0.16749777],
       [0.71259177],
       [0.6419442 ],
       [0.9788424 ],
       [0.973998  ],
       [0.93232   ],
       [0.07881288]], dtype=float32)

In [100]:
# Convert prediction probabilities to labels
model_6_preds = tf.squeeze(tf.round(model_6_pred_probs))
model_6_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0.], dtype=float32)>

In [101]:
# Evaluate results of the model
model_6_results = calculate_results(y_true=val_labels,
                                    y_pred=model_6_preds)
model_6_results

{'accuracy': 81.36482939632546,
 'precision': 84.7972972972973,
 'recall': 72.12643678160919,
 'f1': 77.95031055900621}

In [102]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 88.6178861788618,
 'recall': 62.643678160919535,
 'f1': 73.4006734006734}

### Model 7: TF Hub Pretrained USE but with 10% of training data

Transfer learning really helps when you don't have a large dataset.

To see how our model performs on a smaller dataset, let's replicate `model+6` except we'll train it on 10% of the data.

In [104]:
# Create subsets of 10% of the training data
train_10_percent = train_df_shuffled[["text", "target"]].sample(frac=0.1, random_state=42)
train_sentences_10_percent = train_10_percent["text"].to_list()
train_labels_10_percent = train_10_percent["target"].to_list()
len(train_sentences_10_percent), len(train_labels_10_percent)

(761, 761)

In [106]:
# Check the number of target in our subset of data
train_10_percent["target"].value_counts()

target
0    413
1    348
Name: count, dtype: int64

In [108]:
train_df_shuffled["target"].value_counts()

target
0    4342
1    3271
Name: count, dtype: int64

In [110]:
# Create a model
model_7 = tf.keras.Sequential([
    sentence_encoder_layer,
    layers.Dense(64, activation="relu"),
    layers.Dense(1, activation="sigmoid", name="output_layer")
], name="USE_10_percent")

# Compile the model
model_7.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

model_7.summary()

Model: "USE_10_percent"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_9 (Dense)             (None, 64)                32832     
                                                                 
 output_layer (Dense)        (None, 1)                 65        
                                                                 
Total params: 256830721 (979.73 MB)
Trainable params: 32897 (128.50 KB)
Non-trainable params: 256797824 (979.61 MB)
_________________________________________________________________


In [111]:
# Fit the model
model_7_history = model_7.fit(train_sentences_10_percent,
                              train_labels_10_percent,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     "USE_10_percent")])

Saving TensorBoard log files to: model_logs/USE_10_percent/20240719-171148
Epoch 1/5
24/24 [==============================] - 8s 225ms/step - loss: 0.6644 - accuracy: 0.7083 - val_loss: 0.6211 - val_accuracy: 0.7927
Epoch 2/5
24/24 [==============================] - 1s 41ms/step - loss: 0.5870 - accuracy: 0.7898 - val_loss: 0.5433 - val_accuracy: 0.8031
Epoch 3/5
24/24 [==============================] - 1s 43ms/step - loss: 0.5154 - accuracy: 0.8121 - val_loss: 0.4778 - val_accuracy: 0.8150
Epoch 4/5
24/24 [==============================] - 1s 28ms/step - loss: 0.4624 - accuracy: 0.8068 - val_loss: 0.4363 - val_accuracy: 0.8241
Epoch 5/5
24/24 [==============================] - 1s 44ms/step - loss: 0.4278 - accuracy: 0.8265 - val_loss: 0.4068 - val_accuracy: 0.8386


In [112]:
# Find pred propbs for model 7
model_7_pred_probs = model_7.predict(val_sentences)
model_7_pred_probs[:10]

24/24 [==============================] - 1s 19ms/step


array([[0.22515227],
       [0.80286455],
       [0.8940527 ],
       [0.29356202],
       [0.8017386 ],
       [0.8106487 ],
       [0.8933159 ],
       [0.92385733],
       [0.8196268 ],
       [0.05256459]], dtype=float32)

In [113]:
# Convert pred probs to labels
model_7_preds = tf.squeeze(tf.round(model_7_pred_probs))
model_7_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0.], dtype=float32)>

In [114]:
# Evaluate model
model_7_results = calculate_results(y_true=val_labels,
                                    y_pred=model_7_preds)
model_7_results

{'accuracy': 83.85826771653542,
 'precision': 85.26645768025078,
 'recall': 78.16091954022988,
 'f1': 81.5592203898051}

In [115]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 88.6178861788618,
 'recall': 62.643678160919535,
 'f1': 73.4006734006734}